In [2]:
import numpy as np
import pandas
import scanpy as sc
import anndata
import json
import matplotlib.pyplot as plt
import scvelo as scv
import os
import requests
import shutil
import boto3
import scanpy as sc

# Nascent transcript (intronic reads) analysis

First we load the data

In [3]:
adata = sc.read_loom("/home/ubuntu/jupyter/count_matrices/filtered.loom")

In [4]:
adata.obs.index = [i.split(":")[-1][:-len(".noduplicates.sam")] for i in adata.obs.index]

In [5]:
keys = ["m1", "m3", "m5"]

In [6]:
data = {}
for key in keys:
    data[key] = adata[[key in i for i in adata.obs.index], :]

In [7]:
data

{'m1': View of AnnData object with n_obs × n_vars = 4 × 46904 
     var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
     layers: 'matrix', 'ambiguous', 'spanning', 'spliced', 'unspliced',
 'm3': View of AnnData object with n_obs × n_vars = 4 × 46904 
     var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
     layers: 'matrix', 'ambiguous', 'spanning', 'spliced', 'unspliced',
 'm5': View of AnnData object with n_obs × n_vars = 4 × 46904 
     var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
     layers: 'matrix', 'ambiguous', 'spanning', 'spliced', 'unspliced'}

In [8]:
results = {}
for key, matrix in data.items():
    results_list = results.setdefault(key, [])
    for i in range(matrix.shape[0]):
        result = sorted(zip(matrix.layers["unspliced"].toarray()[i, :], matrix.var.index), reverse=True)
        filtered_result = [i for i in result if i[0] > 10]
        results_list.append(filtered_result)

In [9]:
genes_per_celltype = {}
for key in keys:
    gene_sets = []
    for result_list in results[key]:
        gene_sets.append(set([i[1] for i in result_list]))
    genes_per_celltype[key] = set.intersection(*gene_sets)

# Find genes which are consistently intronically expressed across all cells within a group (at a cutoff of >10 intronic reads, and >10 spliced reads)

In [10]:
for key in keys:
    nascent_genes = set()
    print(key)
    for gene in genes_per_celltype[key]:
        spliced_data = data[key].layers["spliced"][:, data[key].var.index == gene].toarray()
        avg_count = (spliced_data.sum(0)/spliced_data.shape[0])[0]
        if avg_count > 10:
            nascent_genes.add(gene)
    print(nascent_genes)
    for gene in nascent_genes:
        print(gene)

m1
{'vha-13', 'ssl-1', 'T21B10.3', 'Y43F8C.6', 'unc-62', 'pqn-80', 'atgl-1', 'hum-2', 'alg-5', 'B0432.8', 'K06B9.2', 'bet-1', 'ZC506.1', 'F40E10.6', 'Y51F10.2', 'unc-73', 'K10D3.4'}
vha-13
ssl-1
T21B10.3
Y43F8C.6
unc-62
pqn-80
atgl-1
hum-2
alg-5
B0432.8
K06B9.2
bet-1
ZC506.1
F40E10.6
Y51F10.2
unc-73
K10D3.4
m3
{'vha-13', 'T21B10.3', 'Y43F8C.6', 'gnrr-2', 'pqn-80', 'atgl-1', 'B0432.8', 'C44E4.5', 'K06B9.2', 'bet-1', 'ZC506.1', 'F40E10.6', 'Y51F10.2', 'unc-73', 'K10D3.4'}
vha-13
T21B10.3
Y43F8C.6
gnrr-2
pqn-80
atgl-1
B0432.8
C44E4.5
K06B9.2
bet-1
ZC506.1
F40E10.6
Y51F10.2
unc-73
K10D3.4
m5
{'vha-13', 'T21B10.3', 'atgl-1', 'B0432.8', 'C44E4.5', 'K06B9.2', 'bet-1', 'ZC506.1', 'F40E10.6', 'unc-73'}
vha-13
T21B10.3
atgl-1
B0432.8
C44E4.5
K06B9.2
bet-1
ZC506.1
F40E10.6
unc-73


# Check if there are any genes whose expression is unique to a given group

Genes present in m5, but not present in m3

In [12]:
genes_per_celltype["m5"].difference(genes_per_celltype["m3"])

{'F18A12.7'}

Genes present in m3, but not present in m5

In [13]:
genes_per_celltype["m3"].difference(genes_per_celltype["m1"])

{'C24A3.1',
 'C44E4.5',
 'D1065.2',
 'D2096.10',
 'Y54G2A.21',
 'ZK673.11',
 'acr-7',
 'his-57',
 'mks-5'}

The other direction

In [14]:
genes_per_celltype["m1"].difference(genes_per_celltype["m3"])

{'F18A12.7', 'alg-5', 'eps-8', 'hum-2', 'ssl-1', 'unc-62'}

In [15]:
genes_per_celltype["m3"].difference(genes_per_celltype["m5"])

{'C24A3.1', 'Y51F10.2', 'acr-7', 'his-57', 'pqn-80', 'rga-9'}

And the extremes

In [16]:
genes_per_celltype["m1"].difference(genes_per_celltype["m5"])

{'Y51F10.2', 'alg-5', 'eps-8', 'hum-2', 'pqn-80', 'rga-9', 'ssl-1', 'unc-62'}

In [17]:
genes_per_celltype["m5"].difference(genes_per_celltype["m1"])

{'C44E4.5', 'D1065.2', 'D2096.10', 'Y54G2A.21', 'ZK673.11', 'mks-5'}

# Pick up the most highly expressed intronic reads, for subsequent checking with DESeq2

In [44]:
high_intronic_genes = set()
for key in keys:
    print(key)
    gene_list = list(genes_per_celltype[key])
    expr_array = data[key][:, gene_list].layers["unspliced"]
    result = sorted(zip((expr_array.sum(0)/expr_array.shape[0]).tolist()[0], gene_list), reverse=True)[:5]
    current_set = set(i[1] for i in result)
    print(result)
    print(current_set)
    high_intronic_genes.update(current_set)

m1
[(298.25, 'plg-1'), (273.75, 'irld-23'), (127.75, 'T21B10.3'), (57.75, 'K06B9.2'), (57.25, 'K10D3.4')]
{'plg-1', 'K06B9.2', 'irld-23', 'T21B10.3', 'K10D3.4'}
m3
[(302.75, 'plg-1'), (299.25, 'irld-23'), (145.0, 'T21B10.3'), (109.25, 'mks-5'), (85.0, 'K10D3.4')]
{'plg-1', 'mks-5', 'irld-23', 'T21B10.3', 'K10D3.4'}
m5
[(265.0, 'irld-23'), (200.0, 'plg-1'), (121.25, 'T21B10.3'), (59.0, 'K06B9.2'), (52.75, 'K10D3.4')]
{'plg-1', 'K06B9.2', 'irld-23', 'T21B10.3', 'K10D3.4'}


The way to read these results would be as follows:

363.0 -- there are on average 363 (unique) reads in the group

Y51F10.2 -- for the gene Y51F10.2

[(363.0, 'Y51F10.2'), (243.5, 'T21B10.3'), (227.0, 'bet-1'), (173.75, 'ssl-1'), (148.75, 'ZC506.1')]

Most importantly, the gene Y51F10.2 stands out. It is completely absent in group M5, and only present, and highly expressef in groups m3 and m1. It appears that mature oocytes turn on transcription of this gene to prepare them for their future fate.

We will use the respective lists of genes to check if they have differential spliced counts between groups using DESeq2.

In [19]:
high_intronic_genes

{'C44E4.5', 'T21B10.3', 'Y51F10.2', 'ZC506.1', 'bet-1', 'ssl-1', 'vha-13'}

# Some further exploratory analysis looking at plg-1

In [45]:
data["m5"][:,'plg-1'].layers["spliced"].toarray()

array([[0],
       [0],
       [9],
       [0]], dtype=uint32)

In [46]:
data["m3"][:,'plg-1'].layers["spliced"].toarray()

array([[1],
       [5],
       [0],
       [0]], dtype=uint32)

In [47]:
data["m1"][:,'plg-1'].layers["spliced"].toarray()

array([[0],
       [0],
       [1],
       [0]], dtype=uint32)

In [48]:
data["m5"][:,'plg-1'].layers["unspliced"].toarray()

array([[107],
       [254],
       [113],
       [326]], dtype=uint32)

In [49]:
data["m3"][:,'plg-1'].layers["unspliced"].toarray()

array([[244],
       [164],
       [172],
       [631]], dtype=uint32)

In [50]:
data["m1"][:,'plg-1'].layers["unspliced"].toarray()

array([[326],
       [138],
       [128],
       [601]], dtype=uint32)

# Further exploratory analysis looking at total expression

In [31]:
data["m5"].layers["spliced"].sum(1)

matrix([[640812],
        [656772],
        [553987],
        [765955]], dtype=uint64)

In [33]:
data["m3"].layers["spliced"].sum(1)

matrix([[ 962905],
        [ 927053],
        [ 991694],
        [1262718]], dtype=uint64)

In [34]:
data["m1"].layers["spliced"].sum(1)

matrix([[ 765955],
        [ 762107],
        [ 711156],
        [1041213]], dtype=uint64)

In [35]:
data["m5"].layers["unspliced"].sum(1)

matrix([[2762],
        [2878],
        [2575],
        [2942]], dtype=uint64)

In [36]:
data["m3"].layers["unspliced"].sum(1)

matrix([[3659],
        [3008],
        [3383],
        [4557]], dtype=uint64)

In [37]:
data["m1"].layers["unspliced"].sum(1)

matrix([[2942],
        [2483],
        [2230],
        [4435]], dtype=uint64)